<a href="https://colab.research.google.com/github/vijayshankarrealdeal/intro_to_pytorch-Gans/blob/main/ConditionalGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Conditional GAN
#input noise vector + one-hot-encoded vector of class

In [3]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)

In [4]:
def show_tensor_image(image_tensor,num_images = 25,size = (1,28,28), nrow=5, show=True):
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=nrow)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    if show:
        plt.show()

In [ ]:
class Generator(nn.Module):
  def __init__(self,input_dim = 10,img_chan = 1,hidden_units = 64):
    super(Generator,self).__init__()
    self.input_dim = input_dim
    self.gen = nn.Sequential(
        self.make_gen_block(input_dim,hidden_units*4),
        self.make_gen_block(hidden_units*4,hidden_units*2,kernel_size = 4,stride = 1),
        self.make_gen_block(hidden_units*2,hidden_units),
        self.make_gen_block(hidden_units,img_chan,kernel_size = 4,final_layer =True)
    )

  def make_gen_block(self,input_units,output_units,kernel_size = 3, stride = 2,final_layer = False):
    if not final_layer:
      return nn.Sequential(
          nn.ConvTranspose2d(input_units,output_units,kernel_size,stride),
          nn.BatchNorm2d(output_units),
          nn.ReLU(inplace=True),
      )
    else:
      return nn.Sequential(
          nn.ConvTranspose2d(input_units,output_units,kernel_size,stride),
          nn.Tanh()
      )

  def forward(self,noise):
    x = noise.view(len(noise),self.input_dim,1,1)
    return self.gen(x)

In [5]:
x = torch.randn(5,5)
x

tensor([[-1.1258, -1.1524, -0.2506, -0.4339,  0.8487],
        [ 0.6920, -0.3160, -2.1152,  0.3223, -0.1577],
        [ 1.4437,  0.2660,  0.1665,  0.8744, -0.1435],
        [-0.1116, -0.6136,  1.2590,  2.0050,  0.0537],
        [ 0.6181, -0.4128, -0.8411, -2.3160, -0.1023]])

In [14]:
def get_noise(n_samples,input_dim,device = 'cpu'):
  return torch.randn(n_samples,input_dim,device = device)